In [113]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import numpy as np
import pandas.io.json as pd_json
from sqlalchemy import create_engine


In [198]:
jsonfile = pd.read_json("Resources/mergedjson.json")
jsonfile

modifiedjson = jsonfile.drop(['target_fg','target_og','contributed_by','boil_volume', 'method', 'volume','attenuation_level'], axis = 1)

In [199]:
#function to scale srm values based on scale
#https://www.twobeerdudes.com/beer/srm
def srmCat(s):
    if s['srm'] >= 1 and s['srm'] < 3:
        return 'Pale Lager, Pilsner, Witbier, Berliner Weisse'
    elif 3 >= s['srm'] < 4:
        return 'Straw'
    elif 4 >= s['srm'] < 6:
        return 'Pale Gold'
    elif 6 >= s['srm'] < 8:
        return 'Deep Gold'
    elif 8 >= s['srm'] < 10:
        return 'Pale Amber'
    elif 10 >= s['srm'] < 13:
        return 'Medium Amber'
    elif 13 >= s['srm'] < 17:
        return 'Deep Amber'
    elif 17 >= s['srm'] < 20:
        return 'Amber Brown'
    elif 20 >= s['srm'] < 24:
        return 'Brown'
    elif 24 >= s['srm'] < 35:
        return 'Ruby Brown'
    elif 35 >= s['srm'] < 40:
        return 'Deep Brown'
    elif s['srm'] >= 40:
        return 'Black'
    else:
        return 'Undefined'
    
modifiedjson['srm_category'] = modifiedjson.apply(srmCat, axis=1)

In [200]:
#function to scale ibu into 3 categories 
def ibuCat(i):
    if i['ibu'] >= 0 and i['ibu'] < 25:
        return 'Low'
    elif 25 >= i['ibu'] < 38:
        return 'Medium'
    elif i['ibu'] >=40:
        return 'High'
    else:
        return 'Undefined'
    
modifiedjson['ibu_category'] = modifiedjson.apply(ibuCat, axis =1)

In [201]:
#function to scale abv into 3 categories 
def abvCat(a):
    if a['abv'] >= 0 and a['abv'] < 9:
        return 'Low'
    elif 9 >= a['abv'] < 16:
        return 'Medium'
    elif a['abv'] >=16:
        return 'High'
    else:
        return 'Undefined'

modifiedjson['abv_category'] = modifiedjson.apply(abvCat, axis =1)    

In [202]:
beerdata = modifiedjson[['id','name','ibu','ibu_category','srm','srm_category','abv','abv_category', 'ebc','description',
                        'image_url','ingredients','ph','food_pairing', 'tagline','brewers_tips']].set_index('id')
beerdata.to_csv('beerdata.csv')
shortbeerdata = beerdata.drop(columns=['ingredients', 'food_pairing'])
shortbeerdata

,name,ibu,ibu_category,srm,srm_category,abv,abv_category,ebc,description,image_url,ph,tagline,brewers_tips
id,,,,,,,,,,,,,
1,Buzz,60.0,High,10.0,Medium Amber,4.50,Low,20.0,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.4,A Real Bitter Experience.,The earthy and floral aromas from the hops can...
2,Trashy Blonde,41.5,High,15.0,Amber Brown,4.10,Low,15.0,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.4,You Know You Shouldn't,Be careful not to collect too much wort from t...
3,Berliner Weisse With Yuzu - B-Sides,8.0,Low,4.0,Pale Gold,4.20,Low,8.0,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,3.2,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...
4,Pilsen Lager,55.0,High,15.0,Amber Brown,6.30,Low,30.0,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,4.4,Unleash the Yeast Series.,Play around with the fermentation temperature ...
5,Avery Brown Dredge,59.0,High,5.0,Deep Gold,7.20,Low,10.0,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,4.4,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...
6,Electric India,38.0,Undefined,7.5,Pale Amber,5.20,Low,15.0,"Re-brewed as a spring seasonal, this beer â€“ ...",https://images.punkapi.com/v2/6.png,4.4,Vibrant Hoppy Saison.,Source some really good heather honey to get t...
7,AB:12,35.0,Undefined,40.0,Black,11.20,Undefined,80.0,An Imperial Black Belgian Ale aged in old Inve...,https://images.punkapi.com/v2/7.png,5.3,Imperial Black Belgian Ale.,Don't worry too much about controlling the tem...
8,Fake Lager,40.0,High,6.0,Deep Gold,4.70,Low,12.0,Fake is the new black. Fake is where it is at....,https://images.punkapi.com/v2/8.png,4.4,Bohemian Pilsner.,Once the primary fermentation is complete get ...
9,AB:07,30.0,Undefined,42.0,Black,12.50,Undefined,84.0,Whisky cask-aged imperial scotch ale. Beer per...,https://images.punkapi.com/v2/9.png,5.6,Whisky Cask-Aged Scotch Ale.,Authentic heather honey adds a beautiful flora...


In [197]:
# connection_string = "db/beerdata.sqlite"
# engine = create_engine(f'sqlite:///{connection_string}')
# beerdata.to_sql(name='shortbeerdata', con=engine, if_exists='append', index=True)